# 2.2 Data prep

In [17]:
import numpy as np
import pandas as pd

In [6]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-02-car-price/data.csv'

In [7]:
!wget $data

--2025-08-20 15:12:36--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/refs/heads/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.41M  --.-KB/s    in 0.05s   

2025-08-20 15:12:37 (30.3 MB/s) - ‘data.csv’ saved [1475504/1475504]



In [2]:
df = pd.read_csv('data.csv')

NameError: name 'pd' is not defined

Pasar nombres de columnas todo a minusculas y remplazar espacios con _

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

Pasar valores a minusculas y remplazar espacios con _

In [ ]:
strings = list(df.dtypes[df.dtypes == 'object'].index)
strings

In [ ]:
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [ ]:
df.head()

In [ ]:
df.dtypes

# 2.3 Exploratory data analysis

In [ ]:
for col in df.columns:
    print(col)
    print(df[col].unique()[:5])
    print(df[col].nunique())
    print()

Distribution of price

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Explorando la distribucion del precio... Al hacer el gr'afico con todos los datos, vimos que hay una cola muy larga, hay coches con precios de hasta 2 millones. Por eso, recortamos esa cola y hacemos el grafico solo con datos donde el precio es menor a 100K

In [ ]:
sns.histplot(df.msrp[df.msrp < 100000], bins=50)

Vemos que sigue habiendo una cola larga (the data skews to the right), para hacer el rango de datos mas compacto y simetrico (bell curve), apicamos logaritmo a todos los valores...

In [ ]:
price_logs = np.log1p(df.msrp)

In [ ]:
sns.histplot(price_logs, bins=50)

Al graficar los valores con el logaritmo aplicado, vemos que el rango es mas compacto y la grafica es mas simetrica.

Missing values

checamos cuantos null existen por cada columna

In [ ]:
df.isnull().sum()

# 2.4 Setting up the validation framework

Definir las dimensiones de cada dataset

Train = 60%
Validate = 20%
Test = 20%

In [ ]:
n = len(df)

n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

In [ ]:
n_val, n_test, n_train

Para que los sub-datasets no traigan los registros en secuencia desde el dataset original, hay que generar un array de tamanio n, con numeros de 0 a n-1 randomizados

In [ ]:
idx = np.arange(n)

In [ ]:
np.random.seed(2)
np.random.shuffle(idx)

Traer del dataset original, un conjunto definido por los indexes randomizados (idx[:n_train])

In [ ]:
df_train = df.iloc[idx[:n_train]]
df_val = df.iloc[idx[n_train:n_train+n_val]]
df_test = df.iloc[idx[n_train+n_val:]]

Los indexes de los nuevos datasets, seran los indexes traidos del dataset original, eso no nos sirve, vamos a eliminar esos y ponerlos de 0 al numero definido por el tamanio

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

Como mencionamos arriba, hay que aplicar el logaritmo al precio para compactar el rango de datos y hacerlos simetricos. Tambien, guardamos el precio en su propio array

In [ ]:
y_train = np.log1p(df_train.msrp.values)
y_val = np.log1p(df_val.msrp.values)
y_test = np.log1p(df_test.msrp.values)

Eliminamos el precio del dataset de los features

In [ ]:
del df_train['msrp']
del df_val['msrp']
del df_test['msrp']

In [ ]:
len(y_train)

# 2.5 Linear regression

Un repaso rapido de como funciona la regresion lineal:

In [3]:
xi = [453, 11, 86]

In [10]:
w0 = 7.17
w = [1, 1, 1]

In [5]:
def linear_regression(xi):
    n = len(xi)

    pred = w0

    for j in range(n):
        pred = pred + w[j] * xi[j]

    return pred

In [6]:
linear_regression(xi)

550

In [7]:
w0

0

# 2.6 Linear regression vector form

Primero definimos el dot product de los valores a la derecha del w0 en la formula de la linear reg

In [8]:
def dot(xi, w):
    n = len(xi)

    res = 0.0

    for j in range(n):
        res = res + xi[j] * w[j]

    return res

Despues, en la funcion de linear reg, sumamos w0 al dot product anterior

In [9]:
def linear_regression(xi):
    return w0 + dot(xi, w)

Va otro ejemplo, pero ahora, vamos a incluir el w0 en el dot product. Al final, para incluir el w0 en el dot product, solamente lo agregamos como otro weight y multiplicandolo por 1, para que quede igual. Entonces la formula de lnear reg, en vez de ser w0 mas el dot product, la podemos simplificar, incluyendo el w0 como parte del dot product. Es decir, el w0 es parte del vector w, en vez de estar afuera.

In [15]:
w0 = 7.17
w = [0.01, 0.04, 0.002]
w_new = [w0] + w

 Creamos un dataset de ejemplo, notar que siempre hay un 1 al inicio, esto es, como dice arriba, para multiplicarlo por el w0 y quede igual.

 Despues, creamos la lista X y la convertimos en un array de listas.

In [18]:
x1 = [1, 148, 24, 1385]
x2 = [1, 132, 25, 2031]
x10 = [1, 453, 11, 86]

X = [x1, x2, x10]
X = np.array(X)
X

array([[   1,  148,   24, 1385],
       [   1,  132,   25, 2031],
       [   1,  453,   11,   86]])

Entonces, podemos llevar a cabo el calculo de la linear reg, solamente con la funcion dot(), ya que el w0 es parte del vector w y el 1, parte del vector x.

In [19]:
def linear_regression(X):
    return X.dot(w_new)

In [20]:
linear_regression(X)

array([12.38 , 13.552, 12.312])